In [3965]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3966]:
template = 'https://www.jobs.bg/front_job_search.php?subm=1&keywords%5B%5D={}'
#template = 'https://www.jobs.bg/front_job_search.php?subm=1&categories%5B0%5D=56&keywords%5B0%5D={}'


In [3967]:
def get_url(position):
    template = 'https://www.jobs.bg/front_job_search.php?subm=1&keywords%5B%5D={}'
    #template = 'https://www.jobs.bg/front_job_search.php?subm=1&categories%5B0%5D=56&keywords%5B0%5D={}'
    url = template.format(position)
    return url

In [3968]:
url = get_url("data analyst")

# Extract raw html

In [3969]:
response = requests.get(url)

In [3970]:
response

<Response [200]>

In [3971]:
response.reason

'OK'

In [3972]:
soup = BeautifulSoup(response.text, 'html.parser')

In [3973]:
cards = soup.find_all('li', {'class': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17, 18, 19, 20,21, 22]})

In [3974]:
len(cards)

20

# Prototype the model with a single record

In [3975]:
card = cards[0]

In [3976]:
atag = card.div.a

In [3977]:
job_title = atag.get('title')

In [3978]:
job_url = atag.get('href')

In [3979]:
company_name = card.find('div', {'class':'secondary-text'}).text.strip()

In [3980]:
job_location = card.find('div', "card-info card__subtitle").text.strip()

In [3981]:
#card.find('div',{'class':'card-tags pt-8'})

In [3982]:
not_needed = r'\n'
if not_needed in date_posted:
    date_posted = card.find('div','mdc-layout-grid__cell mdc-layout-grid__cell--span-12 flex').text.strip()[0:4]
else:
    date_posted = card.find('div','mdc-layout-grid__cell mdc-layout-grid__cell--span-12 flex').text.strip()[0:8]

In [3983]:
today = datetime.today().strftime('%Y-%m-%d')

In [3984]:
try:
    job_salary = card.find('div', 'card-info card__subtitle').text.strip()[26:49]
except AttributeError:
    job_salary = ''

# Generalize the model with a function

In [3985]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

UnboundLocalError: local variable 'date_posted' referenced before assignment

In [ ]:
records[0:19]

# Getting the next page

In [ ]:
#soup.find('ul', 'page-2')

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('li', {'class': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17, 18, 19, 20]})

for card in cards:
    record = get_record(card)
    records.append(record)

In [ ]:
with open('job_data.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'CompanyName','JobLocation','PostDate','ExtractDate','JobSalary','JobUrl'])
        writer.writerows(records)

# Putting it all together

# import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

def get_url(position):
    template = 'https://www.jobs.bg/front_job_search.php?subm=1&categories%5B0%5D=56&keywords%5B0%5D={}'
    url = template.format(position)
    return url

def get_record(card):
    """Extract job data from a single record"""
    atag = card.div.a
    job_title = atag.get('title')
    job_url = atag.get('href')
    company_name = card.find('div', {'class':'secondary-text'}).text.strip()
    job_location = card.find('div', {'class':'card-info card__subtitle'}).text.strip()[0:5]
    if date_posted.contains('\n'):
        date_posted = card.find('div','mdc-layout-grid__cell mdc-layout-grid__cell--span-12 flex').text.strip()[0:4]
    else:
        date_posted = card.find('div','mdc-layout-grid__cell mdc-layout-grid__cell--span-12 flex').text.strip()[0:8]
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span', 'white-space: nowrap')
    except AttributError:
        job_salary = ''
        
    record = (job_title, company_name, job_location, date_posted, today, job_salary, job_url)
    
    return record

def main(position):
    records = []
    url = get_url(position)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('li', {'class': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17, 18, 19, 20]})

        for card in cards:
            record = get_record(card)
            records.append(record)
    # save data
    with open('results_job_data.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'CompanyName','JobLocation','PostDate','ExtractDate','JobSalary','JobUrl'])
        writer.writerows(records)

In [ ]:
# run main function
#main("Analyst")

In [ ]:
df = pd.read_csv('job_data.csv')

In [3991]:
df = df.loc[(df['JobLocation'] == 'София') | (df['JobLocation'] == 'wifi ')]
#df = datetime.date_posted().strftime('%Y-%m-%d')
#df[df['JobLocation'].str.contains('София', na=False)]
#df = df[df['JobTitle'].str.contains('Data Analyst')]
#df['PostDate'] = pd.to_datetime(df['PostDate'])


In [3988]:
df

,JobTitle,CompanyName,JobLocation,PostDate,ExtractDate,JobSalary,JobUrl
2,Marketing Data Analyst,АктивТрейдс - клон София КЧТ,София,09.12.22,2022-12-12,NaN,https://www.jobs.bg/job/6621002
3,Business Data Analyst,ИНДЕВР ООД,София,09.12.22,2022-12-12,NaN,https://www.jobs.bg/job/6620726
4,Data Analyst,KBC Global Services BG/Shared Service Center,София,08.12.22,2022-12-12,NaN,https://www.jobs.bg/job/6619660
5,M&A Data Analyst with Turkish language,INTERNET SECURITIES BULGARIA EOOD,София,07.12.22,2022-12-12,NaN,https://www.jobs.bg/job/6618177
6,Junior/Mid Data Analyst & Data Engineer,Adastra Bulgaria Ltd.,wifi,07.12.22,2022-12-12,NaN,https://www.jobs.bg/job/6617074
8,Data Analyst (KYC and AML) - Junior and Senior...,Passcon Ltd / AML RightSource,София,29.11.22,2022-12-12,NaN,https://www.jobs.bg/job/6606582
9,Data Analyst,Ай Ес Джи Би Джи ЕООД,София,28.11.22,2022-12-12,NaN,https://www.jobs.bg/job/6604746
10,KYC Data Analyst with German,Passcon Ltd / AML RightSource,София,28.11.22,2022-12-12,NaN,https://www.jobs.bg/job/6592547
11,SEGA QA Data Analyst,Creative Assembly Sofia / SEGA Black Sea Ltd.,София,24.11.22,2022-12-12,NaN,https://www.jobs.bg/job/6602038
13,Financial Analyst (Private Equity Data Solutions),Дъ Бърджис Груп ЕООД,София,24.11.22,2022-12-12,NaN,https://www.jobs.bg/job/6600494
